In [1]:
#!pip install -q langchain --user --upgrade --force-reinstall --progress-bar off

In [2]:
#!pip install -q openai --progress-bar off

In [3]:
temp = 0.0
num_labels = 17
model_name="ft:gpt-3.5-turbo-0125:personal::945FQ0l1"
# model_name="gpt-3.5-turbo-0301"
# model_name="gpt-3.5-turbo-instruct"
# model_name="gpt-4-0613"

In [4]:
output_dir = "../GPT_fine-tuning/data/output/2024.03.18_03_GPT-3.5-FINE-TUNED_zero-shot-learning/"

final_xlsx_filename = output_dir+f"gpt_1000_texts_{num_labels}sdg_temperature_{temp}_model_name_{model_name}.xlsx"
print(final_xlsx_filename)

classification_report_filename = output_dir+f"gpt_1000_texts_{num_labels}sdg_temperature_{temp}_model_name_{model_name}_classification_report.xlsx"
print(classification_report_filename)

final_xlsx_filename_corrected = output_dir+f"gpt_1000_texts_{num_labels}sdg_temperature_{temp}_model_name_{model_name}_corrected.xlsx"
print(final_xlsx_filename_corrected)

../GPT_fine-tuning/data/output/2024.03.18_03_GPT-3.5-FINE-TUNED_zero-shot-learning/gpt_1000_texts_17sdg_temperature_0.0_model_name_ft:gpt-3.5-turbo-0125:personal::945FQ0l1.xlsx
../GPT_fine-tuning/data/output/2024.03.18_03_GPT-3.5-FINE-TUNED_zero-shot-learning/gpt_1000_texts_17sdg_temperature_0.0_model_name_ft:gpt-3.5-turbo-0125:personal::945FQ0l1_classification_report.xlsx
../GPT_fine-tuning/data/output/2024.03.18_03_GPT-3.5-FINE-TUNED_zero-shot-learning/gpt_1000_texts_17sdg_temperature_0.0_model_name_ft:gpt-3.5-turbo-0125:personal::945FQ0l1_corrected.xlsx


In [5]:
import os
from IPython.display import clear_output
os.environ["OPENAI_API_KEY"] = input("INSERT YOU OPENAI API KEY:")
clear_output()

In [6]:
import pandas as pd
import pickle
from sklearn.metrics import classification_report

In [7]:
import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

from typing import Union
from IPython.display import display, HTML

In [8]:
def perform_classification(text: str, chain: LLMChain) -> str:
    """Perform classification using LangChain model API.

    Parameters
    ----------
    text : str
        Input text.
    chain : LLMChain
        LangChain model.

    Returns
    -------
    str or None
        API Response
    """
    try:
        return chain.run(text=text)
    except Exception as e:
        print(e)
        return None
    
def convert_to_dict(row: str) -> dict:
    try:
        row = row.replace("false", "False")
        row = row.replace("true", "True")
        return ast.literal_eval(row)
    except Exception as e:
        print(e, row)
        return row

In [9]:
llm = ChatOpenAI(temperature=temp, model_name=model_name, max_tokens=500)

/home/jovyan/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# Caricamento dati e preparazione prompt


In [10]:
# IN QUESTO ESPERIMENTO CONSIDERO LO STESSO DATASET UTILIZZATO IN 
# GPT_zero-shot_learning/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.ipynb
# IN MODO CHE I DATI SIANO GLI STESSI 

# data = pd.read_excel("../GPT_zero-shot_learning/output/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.xlsx")
# # data.index = data.text_id
# # data.text_id = data.index
# data = data.drop(columns=['Unnamed: 0', 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
# data

# data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx")

data = data.drop(columns=['Unnamed: 0']) #, 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124
...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109


In [11]:
# data.shape

In [12]:
# data.groupby('sdg').count()['text']

In [13]:
#data_sel_4_few_shot_examples = pd.read_excel("../GPT_few-shot_learning/data/output/2023.10.19-1_GPT_few-shot-learning_all-17-sdg/data_sel_4_few_shot_examples.xlsx")

In [14]:
#data_sel_4_few_shot_examples

In [15]:
# https://www.un.org/en/exhibits/page/sdgs-17-goals-transform-world#:~:text=GOAL%201%3A%20No%20Poverty%20GOAL,10%3A%20Reduced%20Inequalities%20GOAL%2011

template = """
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes:

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The text to be processed is:

```
{text}
```
"""

In [16]:
prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [17]:
chain = LLMChain(llm=llm, prompt=prompt)

In [18]:
output_datapath = output_dir

In [19]:
# data_sel = data_sel_4_few_shot_examples
# data_sel = data
# data_sel = data[data.sdg==0].sample(10)
data_sel = data #.sample(3)

In [20]:
# results = data_sel.apply(lambda x: perform_classication(x["text"], chain), axis=1)
import time

results = []
for index, row in data_sel.iterrows():
    cnt = index
    result = perform_classification(row["text"], chain)
    results.append((row["text_id"], result))
    
    result_extended = (cnt, row["text_id"], row["sdg"], result)

    print(result_extended)

    with open(output_datapath+f'/sdg_classification_GPT_{cnt}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)
    
    time.sleep(5)

/home/jovyan/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


(0, 6668, 0, 'SDG 0')
(1, 6099, 0, 'SDG 0')
(2, 6471, 0, 'SDG 0')
(3, 1786, 0, 'SDG 0')
(4, 2143, 0, 'SDG 0')
(5, 3377, 0, 'SDG 0')
(6, 5589, 0, 'SDG 0')
(7, 5431, 0, 'SDG 7')
(8, 768, 0, 'SDG 0')
(9, 2735, 0, 'SDG 0')
(10, 1306, 0, 'SDG 0')
(11, 3844, 0, 'SDG 0')
(12, 1524, 0, 'SDG 0')
(13, 4660, 0, 'SDG 0')
(14, 5077, 0, 'SDG 0')
(15, 995, 0, 'SDG 0')
(16, 2166, 0, 'SDG 0')
(17, 2512, 0, 'SDG 0')
(18, 2191, 0, 'SDG 0')
(19, 6073, 0, 'SDG 0')
(20, 1376, 0, 'SDG 0')
(21, 3889, 0, 'SDG 0')
(22, 6576, 0, 'SDG 0')
(23, 3912, 0, 'SDG 0')
(24, 1565, 0, 'SDG 0')
(25, 1033, 0, 'SDG 0')
(26, 5653, 0, 'SDG 0')
(27, 940, 0, 'SDG 0')
(28, 3613, 0, 'SDG 0')
(29, 1659, 0, 'SDG 0')
(30, 848, 0, 'SDG 0')
(31, 2157, 0, 'SDG 15')
(32, 3604, 0, 'SDG 0')
(33, 3940, 0, 'SDG 0')
(34, 6228, 0, 'SDG 0')
(35, 3148, 0, 'SDG 0')
(36, 2298, 0, 'SDG 0')
(37, 4226, 0, 'SDG 0')
(38, 2335, 0, 'SDG 0')
(39, 4832, 0, 'SDG 0')
(40, 6153, 0, 'SDG 0')
(41, 6772, 0, 'SDG 0')
(42, 6340, 0, 'SDG 0')
(43, 4505, 0, 'SDG 0')
(

(349, 6718, 5, 'SDG 5')
(350, 5449, 5, 'SDG 5')
(351, 6086, 5, 'SDG 5')
(352, 6002, 5, 'SDG 5')
(353, 3703, 5, 'SDG 5')
(354, 5927, 5, 'SDG 5')
(355, 4613, 5, 'SDG 5')
(356, 1510, 5, 'SDG 5')
(357, 747, 5, 'SDG 5')
(358, 630, 5, 'SDG 5')
(359, 5175, 5, 'SDG 5')
(360, 241, 6, 'SDG 6')
(361, 372, 6, 'SDG 6')
(362, 4479, 6, 'SDG 6')
(363, 1568, 6, 'SDG 6')
(364, 6194, 6, 'SDG 6')
(365, 650, 6, 'SDG 6')
(366, 4828, 6, 'SDG 6')
(367, 3495, 6, 'SDG 6')
(368, 2809, 6, 'SDG 6')
(369, 4972, 6, 'SDG 6')
(370, 3653, 6, 'SDG 6')
(371, 379, 6, 'SDG 6')
(372, 6287, 6, 'SDG 6')
(373, 6555, 6, 'SDG 6')
(374, 6626, 6, 'SDG 6')
(375, 3955, 6, 'SDG 6')
(376, 4253, 6, 'SDG 6')
(377, 5477, 6, 'SDG 6')
(378, 3247, 6, 'SDG 6')
(379, 598, 6, 'SDG 6')
(380, 4015, 6, 'SDG 6')
(381, 961, 6, 'SDG 6')
(382, 6717, 6, 'SDG 6')
(383, 1321, 6, 'SDG 6')
(384, 1347, 6, 'SDG 6')
(385, 827, 6, 'SDG 6')
(386, 4027, 6, 'SDG 6')
(387, 6358, 6, 'SDG 14')
(388, 3208, 6, 'SDG 6')
(389, 5966, 6, 'SDG 6')
(390, 4536, 6, 'SDG 6')


(686, 3571, 11, 'SDG 11')
(687, 3211, 11, 'SDG 11')
(688, 3863, 11, 'SDG 11')
(689, 4063, 11, 'SDG 11')
(690, 5806, 11, 'SDG 11')
(691, 5312, 11, 'SDG 11')
(692, 3904, 11, 'SDG 11')
(693, 3444, 11, 'SDG 11')
(694, 1519, 11, 'SDG 11')
(695, 1653, 11, 'SDG 11')
(696, 1900, 11, 'SDG 11')
(697, 2927, 11, 'SDG 11')
(698, 3494, 11, 'SDG 11')
(699, 3673, 11, 'SDG 11')
(700, 3547, 11, 'SDG 11')
(701, 5527, 11, 'SDG 11')
(702, 4443, 11, 'SDG 11')
(703, 926, 11, 'SDG 11')
(704, 5272, 11, 'SDG 11')
(705, 2640, 11, 'SDG 11')
(706, 2678, 11, 'SDG 11')
(707, 1725, 11, 'SDG 11')
(708, 2798, 11, 'SDG 11')
(709, 172, 11, 'SDG 11')
(710, 2226, 11, 'SDG 11')
(711, 1560, 11, 'SDG 11')
(712, 3315, 11, 'SDG 11')
(713, 5109, 11, 'SDG 11')
(714, 5983, 11, 'SDG 11')
(715, 4398, 11, 'SDG 13')
(716, 3172, 11, 'SDG 11')
(717, 148, 11, 'SDG 11')
(718, 4937, 11, 'SDG 11')
(719, 6039, 11, 'SDG 11')
(720, 5667, 12, 'SDG 12')
(721, 5041, 12, 'SDG 12')
(722, 3004, 12, 'SDG 12')
(723, 2110, 12, 'SDG 12')
(724, 6, 12, 'S

(1004, 57, 16, 'SDG 16')
(1005, 339, 16, 'SDG 16')
(1006, 4967, 16, 'SDG 16')
(1007, 1794, 16, 'SDG 5')
(1008, 1864, 16, 'SDG 16')
(1009, 973, 16, 'SDG 16')
(1010, 535, 16, 'SDG 16')
(1011, 2370, 16, 'SDG 11')
(1012, 2870, 16, 'SDG 16')
(1013, 6330, 16, 'SDG 16')
(1014, 2723, 16, 'SDG 16')
(1015, 4695, 16, 'SDG 16')
(1016, 6628, 16, 'SDG 16')
(1017, 4563, 16, 'SDG 16')
(1018, 6710, 16, 'SDG 0')
(1019, 1947, 16, 'SDG 16')


In [21]:
results

[(6668, 'SDG 0'),
 (6099, 'SDG 0'),
 (6471, 'SDG 0'),
 (1786, 'SDG 0'),
 (2143, 'SDG 0'),
 (3377, 'SDG 0'),
 (5589, 'SDG 0'),
 (5431, 'SDG 7'),
 (768, 'SDG 0'),
 (2735, 'SDG 0'),
 (1306, 'SDG 0'),
 (3844, 'SDG 0'),
 (1524, 'SDG 0'),
 (4660, 'SDG 0'),
 (5077, 'SDG 0'),
 (995, 'SDG 0'),
 (2166, 'SDG 0'),
 (2512, 'SDG 0'),
 (2191, 'SDG 0'),
 (6073, 'SDG 0'),
 (1376, 'SDG 0'),
 (3889, 'SDG 0'),
 (6576, 'SDG 0'),
 (3912, 'SDG 0'),
 (1565, 'SDG 0'),
 (1033, 'SDG 0'),
 (5653, 'SDG 0'),
 (940, 'SDG 0'),
 (3613, 'SDG 0'),
 (1659, 'SDG 0'),
 (848, 'SDG 0'),
 (2157, 'SDG 15'),
 (3604, 'SDG 0'),
 (3940, 'SDG 0'),
 (6228, 'SDG 0'),
 (3148, 'SDG 0'),
 (2298, 'SDG 0'),
 (4226, 'SDG 0'),
 (2335, 'SDG 0'),
 (4832, 'SDG 0'),
 (6153, 'SDG 0'),
 (6772, 'SDG 0'),
 (6340, 'SDG 0'),
 (4505, 'SDG 0'),
 (1776, 'SDG 0'),
 (1529, 'SDG 0'),
 (1838, 'SDG 0'),
 (1427, 'SDG 0'),
 (2023, 'SDG 0'),
 (854, 'SDG 0'),
 (2541, 'SDG 0'),
 (6224, 'SDG 0'),
 (6243, 'SDG 0'),
 (4104, 'SDG 0'),
 (1497, 'SDG 0'),
 (2055, 'SDG 0

In [22]:
# results_processed = results.apply(convert_to_dict).apply(pd.Series).loc[:, ["motivation_for_the_answer", "detected_goals", "selected_goal"]]
# results_processed

In [23]:
# results_processed2 = results.apply(convert_to_dict).apply(pd.Series).loc[:, ["goal_label", "motivation_for_the_answer", "sdg_probability"]]
# results_processed2

In [24]:
results_processed = pd.DataFrame(results, columns=['text_id', 'result'])
results_processed['label_pred'] = results_processed['result'].str.extract(r'SDG (\d+)')
results_processed

,text_id,result,label_pred
0,6668,SDG 0,0
1,6099,SDG 0,0
2,6471,SDG 0,0
3,1786,SDG 0,0
4,2143,SDG 0,0
...,...,...,...
1015,4695,SDG 16,16
1016,6628,SDG 16,16
1017,4563,SDG 16,16
1018,6710,SDG 0,0


In [25]:
# final = data_sel.merge(results_processed, left_index=True, right_index=True)
final = data_sel.merge(results_processed, left_on='text_id', right_on='text_id')
# final = data_sel.merge(results_processed2, left_index=True, right_index=True)
# final = final.drop(columns='Unnamed: 0')
final

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,result,label_pred
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,SDG 0,0
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,SDG 0,0
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,SDG 0,0
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,SDG 0,0
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,SDG 0,0
...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,SDG 16,16
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,SDG 16,16
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,SDG 16,16
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,SDG 0,0


In [26]:
final_xlsx_filename

'../GPT_fine-tuning/data/output/2024.03.18_03_GPT-3.5-FINE-TUNED_zero-shot-learning/gpt_1000_texts_17sdg_temperature_0.0_model_name_ft:gpt-3.5-turbo-0125:personal::945FQ0l1.xlsx'

In [27]:
final.to_excel(final_xlsx_filename)

In [28]:
final = pd.read_excel(final_xlsx_filename)

In [29]:
final_corrected = final

In [30]:
import numpy as np
y_true = final_corrected.sdg 
y_pred = final_corrected.label_pred
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        60
           1       0.76      0.90      0.82        60
           2       0.93      0.92      0.92        60
           3       0.94      0.98      0.96        60
           4       0.90      0.95      0.93        60
           5       0.92      0.97      0.94        60
           6       1.00      0.95      0.97        60
           7       0.92      0.93      0.93        60
           8       0.80      0.75      0.78        60
           9       0.87      0.90      0.89        60
          10       0.87      0.65      0.74        60
          11       0.92      0.97      0.94        60
          12       0.97      1.00      0.98        60
          13       0.93      0.88      0.91        60
          14       0.97      0.98      0.98        60
          15       0.92      0.98      0.95        60
          16       1.00      0.88      0.94        60

    accuracy    

In [31]:
results_processed_merged = final
results_processed_merged.head(3)

,Unnamed: 0,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,result,label_pred
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,SDG 0,0
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,SDG 0,0
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,SDG 0,0


In [32]:
results_processed_merged = results_processed_merged.rename(columns={'label_pred': 'predicted_label_clean'})

In [33]:
results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if x>16 else x)

In [34]:
import numpy as np
y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        60
           1       0.76      0.90      0.82        60
           2       0.93      0.92      0.92        60
           3       0.94      0.98      0.96        60
           4       0.90      0.95      0.93        60
           5       0.92      0.97      0.94        60
           6       1.00      0.95      0.97        60
           7       0.92      0.93      0.93        60
           8       0.80      0.75      0.78        60
           9       0.87      0.90      0.89        60
          10       0.87      0.65      0.74        60
          11       0.92      0.97      0.94        60
          12       0.97      1.00      0.98        60
          13       0.93      0.88      0.91        60
          14       0.97      0.98      0.98        60
          15       0.92      0.98      0.95        60
          16       1.00      0.88      0.94        60

    accuracy    

In [35]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
# report_df.to_excel(classification_report_filename)


In [36]:
classification_report_filename

'../GPT_fine-tuning/data/output/2024.03.18_03_GPT-3.5-FINE-TUNED_zero-shot-learning/gpt_1000_texts_17sdg_temperature_0.0_model_name_ft:gpt-3.5-turbo-0125:personal::945FQ0l1_classification_report.xlsx'

In [37]:
report_df[['precision', 'recall', 'f1-score']] *= 100

In [38]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [39]:
print(precision, recall, accuracy, f1_score)

91.69333150011089 91.56862745098039 91.56862745098039 91.44839774759976


In [40]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

91.7 91.6 91.6 91.4
